# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from math import sin, cos, sqrt, atan2, radians
# Module to reverse geocode hotel data. Install with "pip install geopy"
import reverse_geocoder as rg

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_pd = pd.read_csv("output_data/output.csv")
cities_pd.head()

,City,ID,Lat,Long,Max Temp,Humidity,Cloudiness,Wind,Country,Date
0,atuona,4020109,-9.8000,-139.0333,79.29,77,22,18.81,PF,Wed Jun 23 18:37:31 2021
1,plettenberg bay,964712,-34.0527,23.3716,55.27,83,100,4.59,ZA,Wed Jun 23 18:37:31 2021
2,bonavista,5905393,48.6499,-53.1147,64.40,99,99,15.73,CA,Wed Jun 23 18:37:32 2021
3,namatanai,2090021,-3.6667,152.4333,84.25,70,100,3.65,PG,Wed Jun 23 18:37:32 2021
4,broken hill,2173911,-31.9500,141.4333,55.11,58,2,16.11,AU,Wed Jun 23 18:37:32 2021


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(g_key)

In [4]:
locations = cities_pd[["Lat", "Long"]].astype(float)
city_humidity = cities_pd["Humidity"].astype(float) 

In [10]:
heat_max = (cities_pd["Humidity"].max()).astype(float)
print(heat_max)

100.0


In [11]:
center_coord = (40.52,34.34)

fig = gmaps.figure(center=center_coord,zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=city_humidity, 
                                 dissipating=False, max_intensity=heat_max,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
cities_df = pd.DataFrame(cities_pd)
cities_df

,City,ID,Lat,Long,Max Temp,Humidity,Cloudiness,Wind,Country,Date
0,atuona,4020109,-9.8000,-139.0333,79.29,77,22,18.81,PF,Wed Jun 23 18:37:31 2021
1,plettenberg bay,964712,-34.0527,23.3716,55.27,83,100,4.59,ZA,Wed Jun 23 18:37:31 2021
2,bonavista,5905393,48.6499,-53.1147,64.40,99,99,15.73,CA,Wed Jun 23 18:37:32 2021
3,namatanai,2090021,-3.6667,152.4333,84.25,70,100,3.65,PG,Wed Jun 23 18:37:32 2021
4,broken hill,2173911,-31.9500,141.4333,55.11,58,2,16.11,AU,Wed Jun 23 18:37:32 2021
...,...,...,...,...,...,...,...,...,...,...
554,pemangkat,1631637,1.1667,108.9667,78.26,85,98,6.89,ID,Wed Jun 23 18:40:54 2021
555,bentiu,377472,9.2333,29.8333,72.09,81,38,5.30,SS,Wed Jun 23 18:40:55 2021
556,chanika,160592,-5.4167,38.0167,59.13,98,67,2.95,TZ,Wed Jun 23 18:40:55 2021
557,chuguyevka,2025286,44.1653,133.8633,79.27,42,11,3.87,RU,Wed Jun 23 18:40:56 2021


In [13]:
# Ideal weather conditions: 75-85F, Wind Speed 5-25mph, humidity 10-60%

cities_ideal = cities_df[(cities_df['Max Temp'] < 82) & 
            (cities_df['Wind'] >= 5) & (cities_df['Wind'] <= 25) & 
            (cities_df['Humidity'] >= 15) & (cities_df['Humidity'] <= 55)]

cities_ideal_df = pd.DataFrame(cities_ideal)
cities_ideal_df.dropna(inplace=True)
print(len(cities_ideal_df))

47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
cities_ideal_df=cities_ideal_df.drop(columns=["Date"])
cities_ideal_df["Hotel Name"] = ""

In [15]:
def haversine_vectorize(search_lng, search_lat, result_lon, result_lat):

    search_lng, search_lat, result_lon, result_lat = map(np.radians, [search_lng, search_lat, result_lon, result_lat])

    newlon = result_lon - search_lng
    newlat = result_lat - search_lat

    haver_formula = np.sin(newlat/2.0)**2 + np.cos(search_lat) * np.cos(result_lat) * np.sin(newlon/2.0)**2

    dist = 2 * np.arcsin(np.sqrt(haver_formula ))
    distance = 6367 * dist #6367 for distance in KM for miles use 3958
    
    return(distance)

In [16]:
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&fields=formatted_address,name,geometry&locationbias=circle:5000@"

hotel_names = []
hotel_lat = []
hotel_lng = []

# use iterrows to iterate through pandas dataframe
for index, row in cities_ideal_df.iterrows():
    search_city = row["City"]
    search_lat = row["Lat"]
    search_lng = row["Long"]

    # # assemble url and make API request
    response = requests.get(f"{base_url}{search_lat},{search_lng}&key={g_key}").json()
    
    FIELDS = ["formatted_address","geometry.location.lat","geometry.location.lng","name"]
    response_df = pd.json_normalize(response["candidates"])
    response_df = response_df[FIELDS]

    result_lat = response_df["geometry.location.lat"].astype(float)
    result_lon = response_df["geometry.location.lng"].astype(float)
    hotel_name = response_df["name"].astype(str)

    distance = int(haversine_vectorize(result_lon,result_lat,search_lng,
                   search_lat))    
    if distance > 5:
       continue

    hotel_names.append(hotel_name)
    hotel_lat.append(result_lat)
    hotel_lng.append(result_lon)

In [17]:
name_edit = list(map(lambda x:x[0],hotel_names)) 
lat_edit = list(map(lambda x:x[0],hotel_lat)) 
lng_edit = list(map(lambda x:x[0],hotel_lng)) 

In [18]:
hotel_info_df = pd.DataFrame(list(zip(name_edit,lat_edit,lng_edit)),columns=["Name","Lat","Lng"])

In [19]:
city_list = []
country_list = []

for row in hotel_info_df.iterrows():
    coordinates = (row[1]["Lat"],row[1]["Lng"])
    result = rg.search(coordinates)
    result_df = pd.DataFrame(result)
    city_list.append(result_df['name'])
    country_list.append(result_df['cc'])

Loading formatted geocoded file...


In [20]:
city_list = list(map(lambda x:x[0],city_list)) 
country_list = list(map(lambda x:x[0],country_list)) 

hotel_df = pd.DataFrame(list(zip(name_edit,city_list,country_list,lat_edit,lng_edit)),columns=["Hotel Name","City","Country","Lat","Lng"])

In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()])

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))